<br/>

$$ \huge{\underline{\textbf{ Playing Atari Games with Deep RL }}} $$

$$ \large{\textbf{MountainCar + DQN + Memory Reply}} $$

<br/>



In [ ]:
def q_learning(env, frames, gamma, eps_decay_steps, eps_target, action_repeat,
               batch_size, model, mem, callback=None, trace=None):
    """Episodic Semi-Gradient Sarsa
    
    Params:
        env - environment
        ep - number of episodes to run
        gamma - discount factor [0..1]
        eps - epsilon-greedy param
        model      - function approximator, already initialised, with methods:
                     eval(state, action) -> float
                     train(state, target) -> None
    """
    def policy(st, model, eps):
        if np.random.rand() > eps:
            q_values = model.eval(np.array([st]))
            return np.argmax(q_values)
        else:
            return env.action_space.sample()
    
    if eps_decay_steps is not None:
        eps_delta = (1-eps_target) / eps_decay_steps
        eps = 1
    else:
        eps = eps_target
        
    # Fill memory buffer using random policy
    while len(mem) < mem.max_len:
        S = env.reset();
        for t_ in itertools.count():
            if t_ % action_repeat == 0:
                A = env.action_space.sample()    # random policy
            S_, R, done, _ = env.step(A)
            mem.append(S, A, R, S_, done)
            if done:
                break
            S = S_

    tts_ = 0                                 # total time step
    for e_ in itertools.count():             # count from 0 to infinity
        
        S = env.reset()
        
        for t_ in itertools.count():         # count from 0 to infinity
            
            if t_ % action_repeat == 0:
                A = policy(S, model, eps)
            
            S_, R, done, _ = env.step(A)
            
            mem.append(S, A, R, S_, done)
            
            if callback is not None:
                callback(e_, t_, S, A, R, done, eps, model, mem, trace)
            
            states, actions, rewards, n_states, dones, _ = mem.get_batch(batch_size)
            targets = model.eval(n_states)
            targets = rewards + gamma * np.max(targets, axis=-1)
            targets[dones] = rewards[dones]                # return of next-to-terminal state is just R
            model.train(states, actions, targets)

            if done:
                break
                
            if tts_ >= frames:
                return
                
            S = S_
            
            if eps > eps_target:
                eps = max(eps - eps_delta, eps_target)
                
            tts_ += 1

In [ ]:
def enjoy_env(env, frames, eps, action_repeat, model, callback=None, trace=None, render=True, sleep=0):

    def policy(st, model, eps):
        if np.random.rand() > eps:
            q_values = model.eval(np.array([st]))
            return np.argmax(q_values)
        else:
            return env.action_space.sample()
    
    tts_ = 0                                 # total time step
    for e_ in itertools.count():             # count from 0 to infinity
        
        S = env.reset()
        
        if render:
            env.render()
            time.sleep(sleep)
        
        for t_ in itertools.count():         # count from 0 to infinity
            
            if t_ % action_repeat == 0:
                A = policy(S, model, eps)
            
            S_, R, done, _ = env.step(A)
            
            if render:
                env.render()
                time.sleep(sleep)
            
            if callback is not None:
                callback(e_, t_, S, A, R, done, eps, model, mem, trace)
    
            if done:
                break
                
            if tts_ >= frames:
                return
                
            S = S_
                
            tts_ += 1

---

# Experiment Setup

Imports (source file: [tiles3.py](tiles3.py), [helpers_1001.py](helpers_1001.py))

In [ ]:
import pdb

In [ ]:
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tables
import itertools
import collections

import gym

In [ ]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    devs = sess.list_devices()
    print('\n'.join([x.name for x in devs]))

Environment

In [ ]:
import helpers
import importlib
importlib.reload(helpers)

We will need callback to capture q-value array for whole state-action space at specified episodes.

In [ ]:
class Trace():
    def __init__(self, tf_summary_writer, eval_every, render=False, test_states=None, test_labels=None, state_labels=None):
        if test_states is not None:
            assert test_states.ndim == 2
            assert len(test_states) == len(test_labels)
            
        self.tf_summary_writer = tf_summary_writer
        
        
        self.eval_every = eval_every
        self.render = render
        self.test_states = test_states
        self.test_labels = test_labels
        self.state_labels = state_labels
        
        self.total_tstep = 0
        
        self.q_values = collections.OrderedDict()
        self.ep_end_idx = collections.OrderedDict()
        self.ep_rewards = collections.defaultdict(float)
        
        self.states = []
        self.actions = []
        self.rewards = []  # t+1
        self.dones = []    # t+1
        self.epsilons = []

In [ ]:
def callback(episode, tstep, st, act, rew_, done_, eps, model, memory, trace):
    """Called from gradient_MC after every episode.
    
    Params:
        episode [int] - episode number
        tstep [int]   - timestep within episode
        model [obj]   - function approximator
        trace [list]  - list to write results to"""
    
    if trace.total_tstep == 0 and memory is not None:
        print('num exits in memory:', np.count_nonzero(memory._hist_done_1))
        
    if done_:
        trace.ep_end_idx[episode] = trace.total_tstep
    
    trace.states.append(st)
    trace.actions.append(act)
    trace.rewards.append(rew_)
    trace.dones.append(done_)
    trace.epsilons.append(eps)
    
    trace.ep_rewards[episode] += rew_
    
    if done_ and trace.tf_summary_writer is not None:
        summary = tf.Summary()
        summary.value.add(tag='episode_reward', simple_value=trace.ep_rewards[episode])
        trace.tf_summary_writer.add_summary(summary, trace.total_tstep)
    
    if trace.render:
        env.render()
            
    if trace.eval_every is not None:
        if trace.total_tstep % trace.eval_every == 0:
            print()
            print('■'*80)
            print('episode:', episode, '\t time step:', tstep,
                  '\t total time step:', trace.total_tstep, '\t eps:', round(eps,3), 
                  '\t wall time:', datetime.datetime.now())

            
        if trace.total_tstep % trace.eval_every == 0:
            
            if len(st) == 2:
                # We are working with 2D environment,
                # plot whole Q-Value functions across whole state space
            
                q_arr = helpers.eval_state_action_space(model, env, split=[128,128])
                trace.q_values[trace.total_tstep] = q_arr

                helpers.plot_2d_environment(env, trace.total_tstep, 1000, trace, memory,
                                            axis_labels=['state[0]', 'state[1]'],
                                            action_labels=['Act 0', 'Act 1', 'Act 2'],
                                            action_colors=['red', 'blue', 'green'])
                
            else:
                # Environment is not 2D, so we can't plot whole Q-Value function
                # Instead we plot state on standard graph, which is still better than nothing
                
                if trace.test_states is not None:
                    y_hat = model.eval(trace.test_states)
                    trace.q_values[trace.total_tstep] = y_hat
                
                helpers.plot_generic_environment(env, trace.total_tstep, 1000, trace, memory)

    trace.total_tstep += 1

---

# Lunar Lander

In [ ]:
def experiment_lunarlander_tf(frames):
    
    tf_model = TFNeuralNet(nb_in=8, nb_hid_1=64, nb_hid_2=64, nb_out=4, lr=0.00025)
    model = TFFunctApprox(tf_model,
                          np.array([-1., -1., -1., -1., -1., -1., -1., -1.]),  # state space low
                          np.array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
                          rew_mean=0,
                          rew_std=1,
                          nb_actions=env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(8,), state_dtype=float)
    
    trace = Trace(eval_every=1000,
                 test_states=np.array([[0, 1.4, 0, 0, 0, 0, 0, 0],     # init
                                       [0, 0.7, 0, 0, 0, 0, 0, 0],     # half way, no tilt
                                       [0, 0.0, 0, 0, 0, 0, 0, 0],]),  # landing pad
                 test_labels=['start', 'half-way', 'landing-pad'],
                 state_labels=['Pos.x', 'Pos.y', 'Vel.x', 'Vel.y', 'Angle', 'Ang. Vel', 'Left Leg', 'Right Leg'])
    
    if frames != 0:
        with tables.open_file('outarray.h5', mode='w') as f_:
            model._model.setup_logdb(f_)
            q_learning(env, frames=frames, gamma=.99, eps_decay_steps=50000, eps_target=0.1, action_repeat=4,
                       batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    return trace, model, mem

In [ ]:
# Create, don't train
env = gym.make('LunarLander-v2')
trace_rl, model, mem = experiment_lunarlander_tf(frames=0)

In [ ]:
# Train from scratch
env = gym.make('LunarLander-v2')
trace_rl, model, mem = experiment_lunarlander_tf(frames=200000)

In [ ]:
# Save weights
model._model.save('./tf_models/LunarLander-v2.ckpt')

In [ ]:
# Load weights
model._model.load('./tf_models/LunarLander-v2.ckpt')

In [ ]:
# Enjoy agent
try:
    enjoy_env(env, frames=float('inf'), eps=0.0, action_repeat=4, model=model, render=True, sleep=0.0)
except KeyboardInterrupt:
    pass
finally:
    env.close()

# Acrobat Tensorflow

In [ ]:
def experiment_acrobot_tf(frames):
    
    tf_model = TFNeuralNet(nb_in=6, nb_hid_1=64, nb_hid_2=64, nb_out=3, lr=0.00025)
    model = TFFunctApprox(tf_model,
                          env.observation_space.low,
                          env.observation_space.high,
                          rew_mean=-70,
                          rew_std=10,
                          nb_actions=env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(6,), state_dtype=float)
    
    trace = Trace(eval_every=1000)
    
    with tables.open_file('outarray.h5', mode='w') as f_:
        model._model.setup_logdb(f_)
        q_learning(env, frames=frames, gamma=.99, eps_decay_steps=20000, eps_target=0.1, action_repeat=4,
                   batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    #test_car(env, ep=20, model=model, callback=callback, trace=trace)
    print()
    return trace, model, mem

In [ ]:
env = gym.make('Acrobot-v1')
trace_rl, model, mem = experiment_acrobot_tf(frames=25000)

In [ ]:
enjoy_env(env, frames=500, eps=0.1, action_repeat=1, model=model, render=True, sleep=0.03)
env.close()

# Cartpole TensorFlow

In [ ]:
def experiment_cartpole_tf(frames):
    
    # Note: This environment has peculiar dynamics where positive reward,
    # along with non-corrected maximisation bias in Q-Function (we don't use Dual-DQN)
    # causes runaway Q-Value effect where Q-Values increase continously
    # This is ok for now, because RELATIVE Q-Values are still good enough
    # I'm pretty sure using target network along with Dual-DQN is a correct proper fix for this
    
    # CartPole original low/high values for observatios are not very reasonable, so we replace them:
    # env.observation_space.low: [-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38]
    
    tf.reset_default_graph()
    session = tf.Session()
    summary_writer = tf.summary.FileWriter(logdir='tf_log_2/cartpole/1')

    neural_net = TFNeuralNet(tf_session=session, tf_summary_writer=summary_writer,
                             nb_in=4, nb_hid_1=64, nb_hid_2=64, nb_out=2, lr=0.0001)
    
    model = TFFunctApprox(neural_net,
                          st_low=np.array([-.2, -1.0, -0.15, -1.0]),
                          st_high=np.array([.2, 1.0, 0.15, 1.0]),
                          rew_mean=0,
                          rew_std=1,
                          nb_actions=env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(4,), state_dtype=float)
    
    trace = Trace(tf_summary_writer=summary_writer,
                 eval_every=1000,
                 test_states=np.array([[0, 0, 0, 0]]),
                 test_labels=['start'],
                 state_labels=['Cart Pos', 'Cart Vel', 'Pole Ang', 'Pole Vel'])

    summary_writer.add_graph(session.graph)
    summary_writer.flush()
    session.run(tf.global_variables_initializer())

    
    with tables.open_file('outarray.h5', mode='w') as f_:
        model._model.setup_logdb(f_)
        q_learning(env, frames=frames, gamma=.99, eps_decay_steps=20000, eps_target=0.1, action_repeat=1,
                   batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    return trace, model, mem

In [ ]:
env = gym.make('CartPole-v0')
trace_rl, model, mem = experiment_cartpole_tf(frames=25000)

In [ ]:
enjoy_env(env, frames=500, eps=0.1, action_repeat=1, model=model, render=True, sleep=0.03)
env.close()

# MountainCar - TensorFlow

In [ ]:
def experiment_tf(frames):
    
    tf_model = TFNeuralNet(nb_in=2, nb_hid_1=64, nb_hid_2=64, nb_out=3, lr=0.00025)
    model = TFFunctApprox(tf_model,
                          env.observation_space.low,
                          env.observation_space.high,
                          rew_mean=-50,
                          rew_std=15,
                          nb_actions=env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(2,), state_dtype=float)
    
    trace = Trace(eval_every=1000)
    
    with tables.open_file('outarray.h5', mode='w') as f_:
        model._model.setup_logdb(f_)
        q_learning(env, frames=frames, gamma=.99, eps_decay_steps=20000, eps_target=0.1, action_repeat=4,
                   batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    return trace, model, mem

In [ ]:
# Create, don't train
env = gym.make('MountainCar-v0').env
trace_rl, model, mem = experiment_tf(frames=0)

In [ ]:
# Train from scratch
env = gym.make('MountainCar-v0').env
trace_rl, model, mem = experiment_tf(frames=25000)

In [ ]:
# Save weights
model._model.save('./tf_models/MountainCar.ckpt')

In [ ]:
# Load weights
model._model.load('./tf_models/MountainCar.ckpt')

In [ ]:
# Enjoy agent
try:
    enjoy_env(env, frames=float('inf'), eps=0.0, action_repeat=4, model=model, render=True, sleep=0.0)
except KeyboardInterrupt:
    pass
finally:
    env.close()

# MountainCar - Keras

In [ ]:
def experiment_keras(frames):
    
    keras_model = tf.keras.models.Sequential()
    keras_model.add(tf.keras.layers.Dense(256, 'relu', input_dim=2))
    keras_model.add(tf.keras.layers.Dense(256, 'relu'))
    keras_model.add(tf.keras.layers.Dense(3, 'linear'))
    keras_model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(lr=0.00025))
    
    model = function_approximators.KerasFunctApprox(
        keras_model, env.observation_space.low, env.observation_space.high, env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(2,), state_dtype=float)
    
    trace = Trace(eval_every=1000)
    q_learning(env, frames=frames, gamma=.99, eps_decay_steps=50000, eps_target=0.1, action_repeat=4,
               batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    #test_car(env, ep=20, model=model, callback=callback, trace=trace)
    print()
    return trace

In [ ]:
env = gym.make('MountainCar-v0').env
trace_rl = experiment_keras(frames=100000)

# MountainCar Tiles (batch=16)

In [ ]:
def experiment_tiles_batch(frames):
    
    model = function_approximators.TileCodingFuncApprox(
        env.observation_space.low, env.observation_space.high, env.action_space.n,
        learn_rate=0.3, num_tilings=8, init_val=0)
    
    mem = Memory(max_len=1000, state_shape=(2,), state_dtype=float)
    
    trace = Trace(eval_every=1000)
    
    q_learning(env, frames=frames, gamma=1.0, eps_decay_steps=None, eps_target=0.0, action_repeat=4,
               batch_size=64, model=model, mem=mem, callback=callback, trace=trace)
    
    return trace

In [ ]:
env = gym.make('MountainCar-v0').env
trace_t_b_1 = experiment_tiles_batch(frames=5000)

# MountainCar Tiles (batch=1)

In [ ]:
def experiment_tiles_single(frames):
    
    model = function_approximators.TileCodingFuncApprox(
        env.observation_space.low, env.observation_space.high, env.action_space.n,
        learn_rate=0.3, num_tilings=8, init_val=0)
    
    mem = Memory(max_len=1, state_shape=(2,), state_dtype=float)
    
    trace = Trace(eval_every=1000)
    q_learning(env, frames=frames, gamma=1.0, eps_decay_steps=None, eps_target=0.0, action_repeat=4,
               batch_size=1, model=model, mem=mem, callback=callback, trace=trace)
    
    return trace

In [ ]:
env = gym.make('MountainCar-v0').env
trace_t_s_1 = experiment_tiles_single(frames=20000)

# Pendulum2D - TensorFlow

In [ ]:
from environments import Pendulum2DEnv

In [ ]:
def experiment_pendulum_tf(frames):
    
    tf_model = TFNeuralNet(nb_in=2, nb_hid_1=64, nb_hid_2=64, nb_out=3)
    model = TFFunctApprox(tf_model,
                          env.observation_space.low,
                          env.observation_space.high,
                          rew_mean=-210,
                          rew_std=50,
                          nb_actions=env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(2,), state_dtype=float)
    
    trace = Trace(eval_every=1000)
    
    with tables.open_file('outarray.h5', mode='w') as f_:
        model._model.setup_logdb(f_)
        q_learning(env, frames=frames, gamma=.99, eps_decay_steps=20000, eps_target=0.1, action_repeat=1,
                   batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    #test_car(env, ep=20, model=model, callback=callback, trace=trace)
    print()
    return trace

In [ ]:
env = Pendulum2DEnv()
trace_rl = experiment_pendulum_tf(frames=25000)

# Pendulum2D - Keras

In [ ]:
from environments import Pendulum2DEnv

In [ ]:
def experiment_pendulum_keras(frames):
    
    keras_model = tf.keras.models.Sequential()
    keras_model.add(tf.keras.layers.Dense(256, 'relu', input_dim=2))
    keras_model.add(tf.keras.layers.Dense(256, 'relu'))
    keras_model.add(tf.keras.layers.Dense(3, 'linear'))
    keras_model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(lr=0.00025))
    
    model = function_approximators.KerasFunctApprox(
        keras_model, env.observation_space.low, env.observation_space.high, env.action_space.n)
    
    mem = Memory(max_len=100000, state_shape=(2,), state_dtype=float)
    
    trace = Trace(eval_every=1000)
    q_learning(env, frames=frames, gamma=.99, eps_decay_steps=50000, eps_target=0.1, action_repeat=1,
               batch_size=4096, model=model, mem=mem, callback=callback, trace=trace)
    
    #test_car(env, ep=20, model=model, callback=callback, trace=trace)
    print()
    return trace

In [ ]:
env = Pendulum2DEnv()
trace_rl = experiment_pendulum_keras(frames=100000)

# Pendulum2D - Tiles (batch=1)

In [ ]:
from environments import Pendulum2DEnv

In [ ]:
def experiment_pendulum_tiles_single(frames):
    
    model = function_approximators.TileCodingFuncApprox(
        env.observation_space.low, env.observation_space.high, env.action_space.n,
        learn_rate=0.3, num_tilings=32, init_val=0)
    
    mem = Memory(max_len=1, state_shape=(2,), state_dtype=float)
    
    trace = Trace(eval_every=25000)
    q_learning(env, frames=frames, gamma=1.0, eps_decay_steps=None, eps_target=0.0, action_repeat=1,
               batch_size=1, model=model, mem=mem, callback=callback, trace=trace)
    
    #test_car(env, ep=20, model=model, callback=callback, trace=trace)
    print()
    return trace

In [ ]:
env = Pendulum2DEnv()
experiment_pendulum_tiles_single(frames=100000000)

---

# Function Approximators and Memory

In [ ]:
class TFNeuralNet():
    def __init__(self, tf_session, tf_summary_writer, nb_in, nb_hid_1, nb_hid_2, nb_out, lr):
        
        self._sess = tf_session
        self._summary_writer = tf_summary_writer
        
        self.nb_in = nb_in
        self.nb_hid_1 = nb_hid_1
        self.nb_hid_2 = nb_hid_2
        self.nb_out = nb_out
        
        self._f = None
        self._timestep = 0
        
        
        
        with tf.variable_scope('NeuralNet'):
        
            with tf.variable_scope('zz_Inputs'):
                self._x = tf.placeholder(name='xx', shape=[None, nb_in], dtype=tf.float32)
                self._y = tf.placeholder(name='yy', shape=[None, nb_out], dtype=tf.float32)

                tf.summary.histogram('xx', self._x)
                tf.summary.histogram('yy', self._y)
                
            with tf.variable_scope('Hidden_1'):
                self._W_hid_1 = tf.get_variable('hid_1_W', shape=[nb_in, nb_hid_1], dtype=tf.float32)
                self._b_hid_1 = tf.get_variable('hid_1_b', shape=[nb_hid_1], dtype=tf.float32,
                                                initializer=tf.zeros_initializer())
                self._z_hid_1 = tf.matmul(self._x, self._W_hid_1) + self._b_hid_1
                self._h_hid_1 = tf.nn.relu(self._z_hid_1)

                tf.summary.histogram('W_hid_1', self._W_hid_1)
                tf.summary.histogram('b_hid_1', self._b_hid_1)
                tf.summary.histogram('z_hid_1', self._z_hid_1)
                
                # Norm Ratio
                self._W_hid_1_bu = tf.get_variable('hid_1_W_bu', trainable=False,
                                                   initializer=self._W_hid_1.initialized_value())
                norm_ratio = tf.norm( self._W_hid_1-self._W_hid_1_bu ) / tf.norm(self._W_hid_1_bu)
                tf.summary.scalar( 'Param_Update_Norm_Ratio', norm_ratio )

            with tf.variable_scope('Hidden_2'):
                self._W_hid_2 = tf.get_variable('hid_2_W', shape=[nb_hid_1, nb_hid_2], dtype=tf.float32)
                self._b_hid_2 = tf.get_variable('hid_2_b', shape=[nb_hid_2], dtype=tf.float32,
                                               initializer=tf.zeros_initializer())
                self._z_hid_2 = tf.matmul(self._h_hid_1, self._W_hid_2) + self._b_hid_2
                self._h_hid_2 = tf.nn.relu(self._z_hid_2)

                tf.summary.histogram('W_hid_2', self._W_hid_2)
                tf.summary.histogram('b_hid_2', self._b_hid_2)
                tf.summary.histogram('z_hid_2', self._z_hid_2)
                
                # Norm Ratio
                self._W_hid_2_bu = tf.get_variable('hid_2_W_bu', trainable=False,
                                                   initializer=self._W_hid_2.initialized_value())
                norm_ratio = tf.norm( self._W_hid_2-self._W_hid_2_bu ) / tf.norm(self._W_hid_2_bu)
                tf.summary.scalar( 'Param_Update_Norm_Ratio', norm_ratio )

            with tf.variable_scope('Output'):
                self._W_out = tf.get_variable('out_W', shape=[nb_hid_2, nb_out], dtype=tf.float32)
                self._b_out = tf.get_variable('out_b', shape=[nb_out],
                                             initializer=tf.zeros_initializer())
                self._y_hat = tf.matmul(self._h_hid_2, self._W_out) + self._b_out

                tf.summary.histogram('W_out', self._W_out)
                tf.summary.histogram('b_out', self._b_out)
                tf.summary.histogram('y_hat', self._y_hat)
                
                # Norm Ratio
                self._W_out_bu = tf.get_variable('out_W_bu', trainable=False,
                                                   initializer=self._W_out.initialized_value())
                norm_ratio = tf.norm( self._W_out-self._W_out_bu ) / tf.norm(self._W_out_bu)
                tf.summary.scalar( 'Param_Update_Norm_Ratio', norm_ratio )


            with tf.variable_scope('zz_Loss'):
                self._loss = tf.losses.mean_squared_error(self._y, self._y_hat)
                tf.summary.scalar('loss', self._loss)

            self._optimizer = tf.train.RMSPropOptimizer(learning_rate=lr)
            #self._optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.025)
            grads_and_vars = self._optimizer.compute_gradients(self._loss)
            
            
            with tf.control_dependencies([tf.assign(self._W_hid_1_bu, self._W_hid_1),
                                          tf.assign(self._W_hid_2_bu, self._W_hid_2),
                                          tf.assign(self._W_out_bu, self._W_out)]):
                self._train_op = self._optimizer.apply_gradients(grads_and_vars)

            self._dW_hid_1 = grads_and_vars[0][0]
            self._db_hid_1 = grads_and_vars[1][0]
            self._dW_hid_2 = grads_and_vars[2][0]
            self._db_hid_2 = grads_and_vars[3][0]
            self._dW_out = grads_and_vars[4][0]
            self._db_out = grads_and_vars[5][0]
            
            with tf.variable_scope('Hidden_1'):
                tf.summary.scalar('Gradient_Norm', tf.norm(self._dW_hid_1))
            with tf.variable_scope('Hidden_2'):
                tf.summary.scalar('Gradient_Norm', tf.norm(self._dW_hid_2))
            with tf.variable_scope('Output'):
                tf.summary.scalar('Gradient_Norm', tf.norm(self._dW_out))

        self._merged_summaries = tf.summary.merge_all(scope='NeuralNet')
        
                
    def setup_logdb(self, filehandle):
        nb_in, nb_hid_1, nb_hid_2, nb_out = self.nb_in, self.nb_hid_1, self.nb_hid_2, self.nb_out
        self._f = filehandle
        self._f.create_earray(self._f.root, 'loss', atom=tables.Float32Atom(), shape=[0] )
        
        self._f.create_earray(self._f.root, 'x', atom=tables.Float32Atom(), shape=[0, 100, nb_in])
        self._f.create_earray(self._f.root, 'y', atom=tables.Float32Atom(), shape=[0, 100, nb_out])
        
        self._f.create_earray(self._f.root, 'hid_1_z', atom=tables.Float32Atom(), shape=[0, 100, nb_hid_1] )
        self._f.create_earray(self._f.root, 'hid_2_z', atom=tables.Float32Atom(), shape=[0, 100, nb_hid_2] )
        self._f.create_earray(self._f.root, 'out_z', atom=tables.Float32Atom(), shape=[0, 100, nb_out] )
        
        self._f.create_earray(self._f.root, 'hid_1_dW', atom=tables.Float32Atom(), shape=[0, nb_in, nb_hid_1] )
        self._f.create_earray(self._f.root, 'hid_1_db', atom=tables.Float32Atom(), shape=[0, nb_hid_1] )
        self._f.create_earray(self._f.root, 'hid_2_dW', atom=tables.Float32Atom(), shape=[0, nb_hid_1, nb_hid_2] )
        self._f.create_earray(self._f.root, 'hid_2_db', atom=tables.Float32Atom(), shape=[0, nb_hid_2] )
        self._f.create_earray(self._f.root, 'out_dW', atom=tables.Float32Atom(), shape=[0, nb_hid_2, nb_out] )
        self._f.create_earray(self._f.root, 'out_db', atom=tables.Float32Atom(), shape=[0, nb_out] )
        
        self._f.create_earray(self._f.root, 'hid_1_W', atom=tables.Float32Atom(), shape=[0, nb_in, nb_hid_1] )
        self._f.create_earray(self._f.root, 'hid_1_b', atom=tables.Float32Atom(), shape=[0, nb_hid_1] )
        self._f.create_earray(self._f.root, 'hid_2_W', atom=tables.Float32Atom(), shape=[0, nb_hid_1, nb_hid_2] )
        self._f.create_earray(self._f.root, 'hid_2_b', atom=tables.Float32Atom(), shape=[0, nb_hid_2] )
        self._f.create_earray(self._f.root, 'out_W', atom=tables.Float32Atom(), shape=[0, nb_hid_2, nb_out] )
        self._f.create_earray(self._f.root, 'out_b', atom=tables.Float32Atom(), shape=[0, nb_out] )

    def backward(self, x, y):
        assert x.ndim == y.ndim == 2
        
        _, merged_summaries, y_hat, z_hid_2, z_hid_1, loss, \
            hid_1_dW, hid_1_db, hid_2_dW, hid_2_db, out_dW, out_db = self._sess.run(
                [self._train_op, self._merged_summaries, self._y_hat, self._z_hid_2, self._z_hid_1, self._loss,
                self._dW_hid_1, self._db_hid_1, self._dW_hid_2, self._db_hid_2, self._dW_out, self._db_out],
                feed_dict={self._x: x, self._y:y})
        
        if self._summary_writer is not None:
            self._summary_writer.add_summary(merged_summaries, self._timestep)
        self._timestep += 1
        
        
        
        if self._f is not None:
            
            hid_1_W, hid_1_b, hid_2_W, hid_2_b, out_W, out_b = self._sess.run(tf.trainable_variables())
            
            self._f.root.loss.append(np.array(loss, ndmin=1))
            
            self._f.root.x.append(np.expand_dims(x[:100], axis=0))
            self._f.root.y.append(np.expand_dims(y[:100], axis=0))
                        
            self._f.root.hid_1_z.append(np.expand_dims(z_hid_1[:100], axis=0))
            self._f.root.hid_2_z.append(np.expand_dims(z_hid_2[:100], axis=0))
            self._f.root.out_z.append(np.expand_dims(y_hat[:100], axis=0))
            
            self._f.root.hid_1_dW.append(np.expand_dims(hid_1_dW, axis=0))
            self._f.root.hid_1_db.append(np.expand_dims(hid_1_db, axis=0))
            self._f.root.hid_2_dW.append(np.expand_dims(hid_2_dW, axis=0))
            self._f.root.hid_2_db.append(np.expand_dims(hid_2_db, axis=0))
            self._f.root.out_dW.append(np.expand_dims(out_dW, axis=0))
            self._f.root.out_db.append(np.expand_dims(out_db, axis=0))
            
            self._f.root.hid_1_W.append(np.expand_dims(hid_1_W, axis=0))
            self._f.root.hid_1_b.append(np.expand_dims(hid_1_b, axis=0))
            self._f.root.hid_2_W.append(np.expand_dims(hid_2_W, axis=0))
            self._f.root.hid_2_b.append(np.expand_dims(hid_2_b, axis=0))
            self._f.root.out_W.append(np.expand_dims(out_W, axis=0))
            self._f.root.out_b.append(np.expand_dims(out_b, axis=0))
        
        return y_hat, loss
    
    def forward(self, x):
        return self._sess.run(self._y_hat, feed_dict={self._x: x})
    
    def save(self, filepath):
        saver = tf.train.Saver()
        saver.saver(self._sess, filepath)
        
    def load(self, filepath):
        saver = tf.train.Saver()
        saver.restore(self._sess, filepath)

In [ ]:
class TFFunctApprox():

    def __init__(self, model, st_low, st_high, rew_mean, rew_std, nb_actions):
        """Q-function approximator using Keras model

        Args:
            model: Keras compiled model
        """
        st_low = np.array(st_low);    st_high = np.array(st_high)
        self._model = model
        
        assert st_low.ndim == 1 and st_low.shape == st_high.shape
        
        if len(st_low) != model.nb_in:
            raise ValueError('Input shape does not match state_space shape')

        if nb_actions != model.nb_out:
            raise ValueError('Output shape does not match action_space shape')

        # normalise inputs
        self._offsets = st_low + (st_high - st_low) / 2
        self._scales = 1 / ((st_high - st_low) / 2)
        
        self._rew_mean = rew_mean
        self._rew_std = rew_std

    def eval(self, states):
        assert isinstance(states, np.ndarray)
        assert states.ndim == 2

        inputs = (states - self._offsets) * self._scales

        y_hat = self._model.forward(inputs)
        
        #return y_hat
        return y_hat*self._rew_std + self._rew_mean

    def train(self, states, actions, targets):
        
        assert isinstance(states, np.ndarray)
        assert isinstance(actions, np.ndarray)
        assert isinstance(targets, np.ndarray)
        assert states.ndim == 2
        assert actions.ndim == 1
        assert targets.ndim == 1
        assert len(states) == len(actions) == len(targets)
        
        targets = (targets-self._rew_mean) / self._rew_std    # decreases range (std>1) to approx -1..1

        inputs = (states - self._offsets) * self._scales
        all_targets = self._model.forward(inputs)             # this range should be small already
        all_targets[np.arange(len(all_targets)), actions] = targets
        self._model.backward(inputs, all_targets)


In [ ]:
class Memory:
    """Circular buffer for DQN memory reply. Fairly fast."""

    def __init__(self, max_len, state_shape, state_dtype):
        """
        Args:
            max_len: maximum capacity
        """
        assert isinstance(max_len, int)
        assert max_len > 0

        self.max_len = max_len                            # maximum length        
        self._curr_insert_ptr = 0                          # index to insert next data sample
        self._curr_len = 0                                 # number of currently stored elements

        state_arr_shape = [max_len] + list(state_shape)

        self._hist_St = np.zeros(state_arr_shape, dtype=state_dtype)
        self._hist_At = np.zeros(max_len, dtype=int)
        self._hist_Rt_1 = np.zeros(max_len, dtype=float)
        self._hist_St_1 = np.zeros(state_arr_shape, dtype=state_dtype)
        self._hist_done_1 = np.zeros(max_len, dtype=bool)

    def append(self, St, At, Rt_1, St_1, done_1):
        """Add one sample to memory, override oldest if max_len reached.

        Args:
            St [np.ndarray]   - state
            At [int]          - action
            Rt_1 [float]      - reward
            St_1 [np.ndarray] - next state
            done_1 [bool]       - next state terminal?
        """
        self._hist_St[self._curr_insert_ptr] = St
        self._hist_At[self._curr_insert_ptr] = At
        self._hist_Rt_1[self._curr_insert_ptr] = Rt_1
        self._hist_St_1[self._curr_insert_ptr] = St_1
        self._hist_done_1[self._curr_insert_ptr] = done_1
        
        if self._curr_len < self.max_len:                 # keep track of current length
            self._curr_len += 1
            
        self._curr_insert_ptr += 1                         # increment insertion pointer
        if self._curr_insert_ptr >= self.max_len:         # roll to zero if needed
            self._curr_insert_ptr = 0

    def __len__(self):
        """Number of samples in memory, 0 <= length <= max_len"""
        return self._curr_len

    def get_batch(self, batch_len):
        """Sample batch of data, with repetition

        Args:
            batch_len: nb of samples to pick

        Returns:
            states, actions, rewards, next_states, next_done, indices
            Each returned element is np.ndarray with length == batch_len
        """
        assert self._curr_len > 0
        assert batch_len > 0

        
        indices = np.random.randint(                   # randint much faster than np.random.sample
            low=0, high=self._curr_len, size=batch_len, dtype=int)

        states = np.take(self._hist_St, indices, axis=0)
        actions = np.take(self._hist_At, indices, axis=0)
        rewards_1 = np.take(self._hist_Rt_1, indices, axis=0)
        states_1 = np.take(self._hist_St_1, indices, axis=0)
        dones_1 = np.take(self._hist_done_1, indices, axis=0)

        return states, actions, rewards_1, states_1, dones_1, indices


    
    def pick_last(self, nb):
        """Pick last nb elements from memory
        
        Returns:
            states, actions, rewards, next_states, done_1, indices
            Each returned element is np.ndarray with length == batch_len
        """
        assert nb <= self._curr_len
        
        start = self._curr_insert_ptr - nb                # inclusive
        end = self._curr_insert_ptr                       # not inclusive
        indices = np.array(range(start,end), dtype=int)   # indices to pick, can be negative
        indices[indices < 0] += self._curr_len            # loop negative to positive
        
        states = np.take(self._hist_St, indices, axis=0)
        actions = np.take(self._hist_At, indices, axis=0)
        rewards_1 = np.take(self._hist_Rt_1, indices, axis=0)
        states_1 = np.take(self._hist_St_1, indices, axis=0)
        dones_1 = np.take(self._hist_done_1, indices, axis=0)
        
        return states, actions, rewards_1, states_1, dones_1, indices
    